In [302]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import re
import os
from pathlib import Path
from bitarray import bitarray
from typing import Dict, List, Tuple
import time
import requests
import collections
from bitarray import bitarray
from typing import List
import math
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [306]:
from hangman_ml import HangMLP, predict_next_letter, load_words, build_bitset_index, build_lacks_index
import torch
# inside __init__
dict_path = r"C:\Users\7ashk\Downloads\final_words.txt"


In [ ]:

import time, re, json, math, string, collections, requests, torch
from urllib.parse import parse_qs
from bitarray import bitarray

from hangman_ml import (
    HangMLP, predict_next_letter, load_words,
    build_bitset_index, build_lacks_index
)

class FastFilter:
    def __init__(self, words):
        self.by_len = collections.defaultdict(list)
        for w in words:
            self.by_len[len(w)].append(w)
        self.has   = build_bitset_index(self.by_len)
        self.lacks = build_lacks_index(self.has)

    def reset(self, length: int):
        self.length = length
        self.live   = bitarray(len(self.by_len[length])); self.live.setall(True)
        self.present, self.absent = set(), set()

    def update(self, pattern: str, last_guess: str | None):
        if self.length not in self.has:
            return

        if last_guess and last_guess in self.has[self.length]:
            ba = self.has[self.length][last_guess] if last_guess in pattern \
                else self.lacks[self.length][last_guess]
            (self.present if last_guess in pattern else self.absent).add(last_guess)
            self.live &= ba

        for ch in self.present: self.live &= self.has[self.length][ch]
        for ch in self.absent:  self.live &= self.lacks[self.length][ch]

class HangmanAPI:
    """
    Opening: 2 length-freq letters
    Midgame: 2 NN letters
    Endgame: regex / n-gram heuristic
    """
    def __init__(self,
                 dict_path=r"C:\\Users\\7ashk\\Downloads\\final_words.txt",
                 session=None, timeout=None, access_token=None):

        # HTTP endpoints
        self.session = session or requests.Session()
        self.timeout, self.access_token = timeout, access_token
        self.hangman_url = "https://trexsim.com/trexsim/hangman"

        # dictionary + indices
        self.words   = load_words(dict_path, 2, 20)
        self.filter  = FastFilter(self.words)

        # pre-computed frequencies
        self.freq_by_len = {L: [l for l,_ in collections.Counter("".join(ws)).most_common()]
                            for L, ws in self.filter.by_len.items()}
        self.global_freq = [l for l,_ in collections.Counter("".join(self.words)).most_common()]

        # n-gram substring bank
        self.n_bank = self._build_n_bank(self.words)

        # neural model (used for moves 3-4)
        self.max_len = 20
        self.model   = HangMLP(inp=self.max_len + 26)
        self.model.load_state_dict(
            torch.load(r"C:\\Users\\7ashk\\Downloads\\hangman_mlp_balanced.pth",
                       map_location="cpu"))
        self.model.eval()

        self._reset_game_state()

    # ── one-time helper ─────────────────────────────────────
    @staticmethod
    def _build_n_bank(words):
        bank = {n: [] for n in range(3, 30)}
        for w in words:
            for n in range(3, min(30, len(w)) + 1):
                for i in range(len(w) - n + 1):
                    bank[n].append(w[i:i + n])
        return bank

    def _reset_game_state(self):
        self.guessed_letters = []
        self.last_pattern    = None
        self.phase_counter   = 0
        self.curr_dict       = self.words[:]
        self.filter.length   = None        

    _VOWELS = set("aeiou")
    def _vowel_ratio(self, pat: str) -> float:
        return sum(ch in self._VOWELS for ch in pat) / len(pat) if pat else 0.0

    def _ngram_guess(self, board: str) -> str:
        clean = board.replace(" ", "").lower().replace("_", ".")
        k     = self.guessed_letters
        vs    = lambda l: l in self._VOWELS and self._vowel_ratio(clean) > 0.55

        def rank(words):
            c = collections.Counter()
            for w in words: c += collections.Counter(set(w))
            return [l for l,_ in c.most_common() if l not in k and not vs(l)]

        self.curr_dict = [w for w in self.curr_dict
                          if len(w) == len(clean) and re.match(clean, w)]
        pick = rank(self.curr_dict)
        if pick: return pick[0]

        pick = rank(self._sub_find(clean))
        if pick: return pick[0]

        for frac in (2, 3):
            L, win = len(clean), max(3, len(clean) // frac)
            bag = []
            for i in range(L - win + 1):
                bag += self._sub_find(clean[i:i + win])
            pick = rank(bag)
            if pick: return pick[0]

        return next(l for l in self.global_freq if l not in k)

    def _sub_find(self, pat: str):
        return [s for s in self.n_bank[len(pat)] if re.match(pat, s)]

    # ── main decision engine ───────────────────────────────
    def guess(self, board: str) -> str:
        pat = board.replace(" ", "").lower()
        if len(pat) not in self.filter.by_len:
            self.filter.by_len[len(pat)] = ["?" * len(pat)]          # dummy token
            self.filter.has[len(pat)]   = {c: bitarray("1") for c in string.ascii_lowercase}
            self.filter.lacks[len(pat)] = {c: bitarray("0") for c in string.ascii_lowercase}

        if self.last_pattern is None:
            self.filter.reset(len(pat))
            self.open_freq_iter = iter(self.freq_by_len.get(len(pat), list("etaoin")))

        prev = self.guessed_letters[-1] if self.guessed_letters else None
        self.filter.update(pat, prev)

        if self.phase_counter < 2:                 
            letter = next(l for l in self.open_freq_iter if l not in self.guessed_letters)

        elif self.phase_counter < 4:               
            letter = predict_next_letter(self.model, pat,
                                         self.guessed_letters, self.max_len)

        else:                                      
            letter = self._ngram_guess(board)

        # ensure valid letter
        if letter not in string.ascii_lowercase:
            letter = next(l for l in self.global_freq if l not in self.guessed_letters)

        self.phase_counter += 1
        self.last_pattern = pat
        return letter

    def start_game(self, practice=True, verbose=True):
        self._reset_game_state()
        res = self.request("/new_game", {"practice":practice})
        if res.get("status")!="approved":
            if verbose: print("Failed to start a new game"); return False
        gid, word, tries = res["game_id"], res["word"], res["tries_remains"]
        if verbose:
            print(f"Successfully start a new game! Game ID: {gid}. "
                  f"# of tries remaining: {tries}. Word: {word}.")
        while tries>0:
            guess = self.guess(word)
            self.guessed_letters.append(guess)
            if verbose: print("Guessing letter:", guess)

            try:
                res = self.request("/guess_letter",
                                   {"request":"guess_letter","game_id":gid,"letter":guess})
            except Exception: continue

            if verbose: print("Sever response:", res)
            status, tries = res["status"], res["tries_remains"]
            if status=="success":
                if verbose: print("Successfully finished game:", gid)
                return True
            elif status=="failed":
                if verbose: print("Failed game:", gid,
                                  "Because of:", res.get("reason","# of tries exceeded!"))
                return False
            else:
                word = res["word"]
        return False  # safety

    def my_status(self):   return self.request("/my_status")

    def request(self, path, args=None, post_args=None, method=None):
        if args is None: args={}
        if post_args is not None: method="POST"
        if self.access_token:
            (post_args or args)["access_token"] = self.access_token
        time.sleep(0.2)
        r = self.session.request(method or "GET",
                                 self.hangman_url+path,
                                 params=args,data=post_args,
                                 timeout=self.timeout,verify=False)
        if 'json' in r.headers['content-type']: return r.json()
        raise RuntimeError("Unexpected response")

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)


In [308]:
api = HangmanAPI(access_token="ef2f30e2361b230ec9c11d612a6ca7", timeout=2000)


In [309]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

Successfully start a new game! Game ID: bf135aad19a1. # of tries remaining: 6. Word: _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'bf135aad19a1', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ e '}
Guessing letter: a
Sever response: {'game_id': 'bf135aad19a1', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ e '}
Guessing letter: r
Sever response: {'game_id': 'bf135aad19a1', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ e '}
Guessing letter: l
Sever response: {'game_id': 'bf135aad19a1', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ _ e '}
Guessing letter: i
Sever response: {'game_id': 'bf135aad19a1', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ i _ _ e '}
Guessing letter: t
Sever response: {'game_id': 'bf135aad19a1', 'status': 'ongoing', 'tries_remains': 2, 'word': '_ _ i _ _ e '}
Guessing letter: s
Sever response: {'game_id': 'bf135aad19a1', 'status': 'ongoing', 'tries_remains': 2, 'word': 's _ i _ _ e '}
Guess

In [ ]:

start_runs, _, _, start_wins = api.my_status()

GAMES = 10
for i in range(1, GAMES + 1):
    api.start_game(practice=1, verbose=False)

end_runs, _, _, end_wins = api.my_status()

played = end_runs - start_runs      
won    = end_wins  - start_wins
rate   = won / played if played else 0.0

print(f"\nBatch summary → {won}/{played} wins  (success rate {rate:.3f})")



Batch summary → 10/10 wins  (success rate 1.000)


In [ ]:

start_runs, _, _, start_wins = api.my_status()

GAMES = 100
for i in range(1, GAMES + 1):
    api.start_game(practice=1, verbose=False)

end_runs, _, _, end_wins = api.my_status()

played = end_runs - start_runs      
won    = end_wins  - start_wins
rate   = won / played if played else 0.0

print(f"\nBatch summary → {won}/{played} wins  (success rate {rate:.3f})")



Batch summary → 88/100 wins  (success rate 0.880)


In [ ]:

start_runs, _, _, start_wins = api.my_status()

GAMES = 100
for i in range(1, GAMES + 1):
    api.start_game(practice=1, verbose=False)

end_runs, _, _, end_wins = api.my_status()

played = end_runs - start_runs      
won    = end_wins  - start_wins
rate   = won / played if played else 0.0

print(f"\nBatch summary → {won}/{played} wins  (success rate {rate:.3f})")



Batch summary → 86/100 wins  (success rate 0.860)


In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    api.start_game(practice=0,verbose=False)
    time.sleep(0.5)

In [312]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.815
